In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multicollinearity-analysis-dataset/salary_data.csv
/kaggle/input/multicollinearity-analysis-dataset/Advertising.csv
/kaggle/input/salary-dataset/Salary_Data (1).csv


In [2]:
df1 = pd.read_csv('/kaggle/input/multicollinearity-analysis-dataset/Advertising.csv')
df2 = pd.read_csv('/kaggle/input/salary-dataset/Salary_Data (1).csv')

In [3]:
df1.head()

,Unnamed: 0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [4]:
df2.head()

,YearsExperience,Age,Salary
0,1.1,21.0,39343
1,1.3,21.5,46205
2,1.5,21.7,37731
3,2.0,22.0,43525
4,2.2,22.2,39891


# What Kind of Problem we have to solve ?

👉 As we can see in Salary Datset we have 2 independent variables [YearsExperience, Age] and dependent variable [Salary] 

* So what we have to do is Based on these 2 indepedent variables we have to predict dependent variable.
* Linear Regression Best-fit line formula : Salary = B0 + B1(Age) + B2(Experiance)
* here , B0= intercept, B1 and B2 are coefficients/slopes

* Now, if you see, there can be a possibility that the age and experience variable themselves have a high correlation value (>90%) i.e. age and experience are internally correlated with each other. This affects the output 'salary', the features 'age' and 'experience' will be almost same thus implying that we are providing the same information to the output feature 'salary' which we want to compute.

This is the problem that we have to resolve.

In [5]:
import pandas as ps
import statsmodels.api as sm

In [6]:
df1.head()

,Unnamed: 0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


# Remove irrelevant column

In [7]:
df1.drop('Unnamed: 0', axis=1,inplace=True)

In [8]:
df1.head(1)

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1


=====================================================================

#### * Looking at our current dataset, TV, Radio and Newspaper are the independent varibales and Sales is the dependent variable
#### * Based on these independent variables values we have to predict the values of dependent variable.

=====================================================================

# Splitting the Data

In [9]:
X = df1.drop('Sales', axis=1)
y = df1['Sales']

In this case, we will use the multiple linear regression technique 'Ordinary Least Squared'
* Equation of Linear Regression best fit line for this dataset is : y = B0 1 + B1(TV) + B2(Radio) + B3(Newspaper)
* Whenever computing OLS : we need to compute B0 also
* But we dont have the B0 value here : So we will add a column for B0 value and all values in that column will be equal to 1
* To add a constant value column for B0 with all values =1, we will use the statsmodel library

In [10]:
X = sm.add_constant(X)
X.head(1)

,const,TV,Radio,Newspaper
0,1.0,230.1,37.8,69.2


Fit an Ordinary Least Squared Model with intercept on TV, Radio and Newspaper. We will again be using the statsmodel library for this as it has the function OLS for creating the model. Inside the OLS method we have to give endog(output feature)and exog values(input features) as parameters

In [11]:
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           1.58e-96
Time:                        06:29:41   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
TV             0.0458      0.001     32.809      0.000       0.043       0.049
Radio          0.1885      0.009     21.893      0.000       0.172       0.206
Newspaper     -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

This summary helps us understand whether there is multicollinearity / high correlation between the independent features.

According to the summary,

B0 = coeff of const = 4.6251

B1 = coeff of TV = 0.0544

[coeff value means that if we change the TV expenditure(i.e. input feature) by 1 unit, the change in sales(i.e. output) will be 0.0544]

B2 = coeff of Radio = 0.1070

B3 = coeff of Newspaper = 0.0003
B3 coeff => << 0.005 => this shows that we are making an unnecessary expenditure on Newspaper. Thus we can reduce that unnecessary expenditure done on Newspaper. Thus while creating the model, we can just drop this feature.
R-squared value = 0.903 => very close to 1 => the model has fitted very well

P value of const = 0

P value of TV = 0
P value of Radio = 0
P value 

=> Except the feature 'Newspaper' (P-value = 0.954) , all the P values are less than 0.05

std error of const = 0.308
std error of TV = 0.001
std error of Radio = 0.008
std error of Newspaper = 0.006
std error => high number(>0.5) if there is multicollinearity among the independent varibles. But here, the std error are small numbers thus indicating there is no multicollinearity among the independent variablesof Newspaper = 0.954

# Plot Independent features in term of correlation

In [12]:
X.corr()

,const,TV,Radio,Newspaper
const,NaN,NaN,NaN,NaN
TV,NaN,1.000000,0.054809,0.056648
Radio,NaN,0.054809,1.000000,0.354104
Newspaper,NaN,0.056648,0.354104,1.000000


Through this table, we can see the correlation values among the various independent features:

* Between TV and Radio => 0.054809
* Between Radio and Newspaper => 0.354104
* Between TV and Newspaper => 0.056648
This implies that none of the correlation values are >0.5. Thus indicating that there is not much correlation between the independent features and thus no multicollinearity issue among the independent features

# Let's Check for Dataset 2

In [13]:
df2.head()

,YearsExperience,Age,Salary
0,1.1,21.0,39343
1,1.3,21.5,46205
2,1.5,21.7,37731
3,2.0,22.0,43525
4,2.2,22.2,39891


In this case the independent features are 'Years of Experience' and the 'Age' and we have to predict the dependent variable 'Salary' based on these two independent features

In [14]:
X = df2.drop('Salary', axis=1)
y = df2['Salary']

In [15]:
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                     323.9
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           1.35e-19
Time:                        06:29:41   Log-Likelihood:                -300.35
No. Observations:                  30   AIC:                             606.7
Df Residuals:                      27   BIC:                             610.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -6661.9872   2.28e+04     -0.292      0.773   -5.35e+04    4.02e+04
YearsExperience  6153.3533   2337.092      2.633      0.014    1358.037    1.09e+04
Age              1836.0136   1285.034      1.429      0.165    -800.659    4472.686
==============================================================================
Omnibus:                        2.695   Durbin-Watson:                   1.711
Prob(Omnibus):                  0.260   Jarque-Bera (JB):                1.975
Skew:                           0.456   Prob(JB):                        0.372
Kurtosis:                       2.135   Cond. No.                         626.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In this scenario, According to the summary,

B0 = coeff of const = -6661.9872

B1 = coeff of YearsofExperience = 6153.3533

[this coeff value means that if we change the YearsOfExperience(i.e. input feature) by 1 unit, the change in salary(i.e. output) will be 6153.3533]

B2 = coeff of Age = 1836.0136

[thus if we change the Age(i.e. input feature) by 1 unit(1 year), the change in salary(i.e. output) will be 6153.3533]
R-squared value = 0.960 => very close to 1 => the model has fitted very well

P value of const = 0.773

P value of YearsOfExperience = 0.014
P value of Age = 0.165
=> for Age => the P-value is >0.05 => Age and YearsOfExperience may have some kind of correlation

std error of const = 0.308
std error of YearsOfExperience = 2337.092
std er

Here we can see the std errors of both YearsOfExperience and Age are very very high, thus indicating that there is a huge Multicollinearity among themror of Age = 1285.034

#### Confirming the multicollinearity between Age and YearsOfExperience by plotting the correlation table

In [16]:
X.corr()

,const,YearsExperience,Age
const,NaN,NaN,NaN
YearsExperience,NaN,1.000000,0.987258
Age,NaN,0.987258,1.000000


With the help of this Correlation Table / Matrix we can imply that age and yearsofexperience have 98% correlation (very highly correlated). This implies that taking one of these features will be more than enough to predict the salary.

Now the Question is which of the input features (YearsOfExperience and Age) to keep and which one to drop for the final prediction of salary
Remedy for this Multicollinearity proble
m:
Solution 1 : Dont do anything, keep things as it is and don't care about multicollinearity and take all the input features to create the model

Solution 2 : Check the P values for Age and YearsOfExperience. P value of Age > P value of YearsOfExperience. Thus drop the 'Age' feature. This will not have much effect on the model as the correlation is about 98%. Thus the whole model can be trained just by considering the feature 'YearsOfExperience'